# Installation and requirements
OTLMOW-Model has no dependencies other than the standard Python libraries. Currently, you need at least Python version 3.6 to use this library. To install the OTL MOW project into your Python project, use pip to install it:

In [1]:
pip install otlmow-model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 42.4 MB/s eta 0:00:00


To upgrade an existing installation use:

In [ ]:
pip install otlmow-model --upgrade

# Basic usage and code examples
To use any of the classes in the class model, instantiate a class after importing it. In this example the Camera class is used. The import statement is needed to find the class within the file structure.

In [ ]:
from otlmow_model.Classes.Onderdeel.Camera import Camera
camera = Camera()

You can also create an instance dynamically by porving the typeURI. In this case you won't need to import the class specifically. Instead you'll import the `dynamic_create_instance_from_uri` function.

In [ ]:
from otlmow_model.Helpers.AssetCreator import dynamic_create_instance_from_uri
camera_type_uri = 'https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Camera'
camera = dynamic_create_instance_from_uri(camera_type_uri)

Alternatively, you can also use `dynamic_create_instance_from_ns_and_name` by providing the namespace and the class name.

In [ ]:
from otlmow_model.Helpers.AssetCreator import dynamic_create_instance_from_ns_and_name
camera = dynamic_create_instance_from_ns_and_name(namespace='onderdeel', class_name='Camera')

Now, this Camera instance can be used by accessing its properties.

In [ ]:
camera.toestand = 'in-gebruik'
camera.naam = 'CAM0001'
camera.opstelhoogte.waarde = 6.0
print(f'The name of the camera is {camera.naam}')

The name of the camera is CAM0001


It's also possible to instantiate an object by providing a dictionary, using the `from_dict` function, which results in the same object as above. The class is created and the attributes are filled with the data from the dicionary. Either use the function from the Camera object after importing it, or use the function from the generic OTLObject class. (example below)

In [ ]:
from otlmow_model.BaseClasses.OTLObject import OTLObject
d = { 'typeURI': 'https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Camera',
    'toestand': 'in-gebruik',
    'naam': 'CAM0001',
    'opstelhoogte': {'waarde': 6.0}}
camera = OTLObject.from_dict(d)

This library supports validation when assiging values to the properties of OTL compliant objects. For example: assigning the string 'True' to a boolean attribute will only raise a warning as the value can be converted and interpreted.

In [ ]:
camera.isPtz = 'True'


In the example below a floating point number is assigned to the boolean attribute. Conversion between these types is not supported resulting in a `CouldNotConvertToCorrectTypeError`.

In [ ]:
camera.isPtz = 20.0

CouldNotConvertToCorrectTypeError: ignored

The camera instance can be printed in a terminal to view the values of its attrbiutes. Attributes without a value are omitted.

In [ ]:
print(camera)

<Camera> object
    typeURI : https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Camera
    isPtz : True
    naam : CAM0001
    opstelhoogte :
        waarde : 6.0
    toestand : in-gebruik


By using the meta_info function and printing it, you can inspect the meta info of the object. For a class this will print out the attributes and their types.

In [ ]:
from otlmow_model.BaseClasses.MetaInfo import meta_info
print(meta_info(camera))

Showing metadata of Camera:
typeURI: https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Camera
definition: Een CCTV-camera, closed-circuit television camera, kortweg camera, produceert beelden of opnames voor bewaking van een regio vanop afstand. Het is een element dat beelden neemt van een locatie en deze doorgeeft naar verschillende partijen om zo de werkelijke situatie te kunnen inschatten vanop afstand. Deze camera kan van het analoge type zijn of digitaal.
attributes:
    assetId (type: DtcIdentificator)
    beeldverwerkingsinstelling (type: DtcCameraBeeldverwerking)
    bestekPostNummer (type: String)
    configBestandAid (type: DtcDocument) <deprecated since 2.3.0>
    datumOprichtingObject (type: Date)
    dnsNaam (type: String)
    geometry (type: WKT)
    heeftAid (type: Boolean) <deprecated since 2.3.0>
    heeftSpitsstrook (type: Boolean) <deprecated since 2.3.0>
    ipAdres (type: DteIPv4Adres)
    isActief (type: Boolean)
    isPtz (type: Boolean)
    merk (type: KlCa

The meta info of attributes is also available.

In [ ]:
print(meta_info(camera, attribute='toestand'))

Showing metadata of toestand:
typeURI: https://wegenenverkeer.data.vlaanderen.be/ns/implementatieelement#AIMToestand.toestand
definition: Geeft de actuele stand in de levenscyclus van het object.
valid values:
    geannuleerd
    gepland
    in-gebruik
    in-ontwerp
    in-opbouw
    overgedragen
    uit-gebruik
    verwijderd


# Extra features
## Relations
Using the RelationValidator it's possible to validate relations between objects. In the example below, the relation between the two types is not a valid relation resulting in False

In [ ]:
from otlmow_model.Classes.Onderdeel.Camera import Camera
from otlmow_model.Classes.Onderdeel.Bevestiging import Bevestiging
from otlmow_model.Classes.Onderdeel.Wegkantkast import Wegkantkast
from otlmow_model.Helpers.RelationValidator import is_valid_relation

camera = Camera()
kast = Wegkantkast()

print(is_valid_relation(source=camera, target=kast, relation_type=Bevestiging))

False


You can also create relations between objects using the `create_relation` function from RelationCreator. This does require the objects to have asset id's.

In [ ]:
from otlmow_model.Helpers.RelationCreator import create_relation
from otlmow_model.Classes.Onderdeel.Verkeersregelaar import Verkeersregelaar
kast.assetId.identificator = 'kast01'
vr = Verkeersregelaar()
vr.assetId.identificator = 'vr01'
relation = create_relation(source=vr, target=kast, relation_type=Bevestiging)
print(relation)

<Bevestiging> object
    typeURI : https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Bevestiging
    assetId :
        identificator : vr01_-_kast01
        toegekendDoor : OTLMOW
    bron :
    bronAssetId :
        identificator : vr01
    doel :
    doelAssetId :
        identificator : kast01


You can now use the other modules to use these objects. For example the [otlmow-converter](/https://github.com/davidvlaminck/OTLMOW-Converter) module allows you to save these objects into a number of  DAVIE compliant files and also load those files into objects.

## Comparing
There are two functions that can be used to compare lists of assets. The usage of these functions will be illustrated here. First, let's create two lists of assets, one with two assets, and the second one with the same assets but where the second asset will be modified.

In [22]:
from otlmow_model.BaseClasses.MetaInfo import meta_info
from otlmow_model.Classes.Onderdeel.Verkeersregelaar import Verkeersregelaar

asset1 = Verkeersregelaar()
asset1.assetId.identificator = 'asset1'
asset1.naam = 'VR1'

asset2 = Verkeersregelaar()
asset2.assetId.identificator = 'asset2'
asset2.naam = 'VR2'
asset2.toestand = 'in-ontwerp'

list1 = [asset1, asset2]

asset3 = Verkeersregelaar()
asset3.assetId.identificator = 'asset2'
asset3.naam = 'VR2'
asset3.toestand = 'in-gebruik'
asset3.regelaartype = 'type-1'

list2 = [asset1, asset3]

Now, both lists can be compared. The first list is the master list. The second list will be compared against that list. Depending on the level you choose, eitehr the full asset that is different will be returned (object level), or a partial asset will be returned that contains the differences (attribute level).

This first example checks on object level and thus the second asset will be fully returned.

In [23]:
from otlmow_model.Helpers.OTLObjectHelper import compare_two_lists_of_objects_object_level
print(compare_two_lists_of_objects_object_level(first_list=list1, second_list=list2))

[<Verkeersregelaar> object
    typeURI : https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Verkeersregelaar
    assetId :
        identificator : asset2
    naam : VR2
    regelaartype : type-1
    toestand : in-gebruik]


The second example check on attribute level. The first asset is identical, but the second has two differences: the attribute toestand has been modified and it now has a valid value for attribute regelaartype. The expected return list will have an object with the id of the second asset, the modified value for toestand and the added value for regelaartype.

In [24]:
from otlmow_model.Helpers.OTLObjectHelper import compare_two_lists_of_objects_attribute_level
print(compare_two_lists_of_objects_attribute_level(first_list=list1, second_list=list2))

[<Verkeersregelaar> object
    typeURI : https://wegenenverkeer.data.vlaanderen.be/ns/onderdeel#Verkeersregelaar
    assetId :
        identificator : asset2
    regelaartype : type-1
    toestand : in-gebruik]
